In [1]:
import pandas as pd

data1 = pd.read_csv("training_set_features.csv")
data = pd.read_csv("training_set_labels.csv")
print(data.info())
print(data.describe())
print(data1.info())
print(data1.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26707 entries, 0 to 26706
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype
---  ------            --------------  -----
 0   respondent_id     26707 non-null  int64
 1   xyz_vaccine       26707 non-null  int64
 2   seasonal_vaccine  26707 non-null  int64
dtypes: int64(3)
memory usage: 626.1 KB
None
       respondent_id   xyz_vaccine  seasonal_vaccine
count   26707.000000  26707.000000      26707.000000
mean    13353.000000      0.212454          0.465608
std      7709.791156      0.409052          0.498825
min         0.000000      0.000000          0.000000
25%      6676.500000      0.000000          0.000000
50%     13353.000000      0.000000          0.000000
75%     20029.500000      0.000000          1.000000
max     26706.000000      1.000000          1.000000
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26707 entries, 0 to 26706
Data columns (total 36 columns):
 #   Column                       No

In [2]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline

# Separate features and target variables
X = data.drop(columns=['xyz_vaccine', 'seasonal_vaccine', 'respondent_id'])
y = data[['xyz_vaccine', 'seasonal_vaccine']]

# Identify categorical and numerical columns
categorical_cols = X.select_dtypes(include=['object', 'category']).columns
numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns

# Create transformers for preprocessing
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ]
)

In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [5]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier


In [6]:
from sklearn.pipeline import Pipeline

model = MultiOutputClassifier(RandomForestClassifier(n_estimators=100, random_state=42))

clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', model)])


In [7]:
clf.fit(X_train, y_train)


ValueError: Found array with 0 feature(s) (shape=(21365, 0)) while a minimum of 1 is required by RandomForestClassifier.

In [8]:
from sklearn.metrics import roc_auc_score

y_pred_proba = clf.predict_proba(X_test)
y_pred_proba = pd.DataFrame({col: [proba[:, 1] for proba in y_pred_proba[i]] for i, col in enumerate(y_test.columns)})

roc_auc_xyz = roc_auc_score(y_test['xyz_vaccine'], y_pred_proba['xyz_vaccine'])
roc_auc_seasonal = roc_auc_score(y_test['seasonal_vaccine'], y_pred_proba['seasonal_vaccine'])

mean_roc_auc = (roc_auc_xyz + roc_auc_seasonal) / 2
print(f'Mean ROC AUC Score: {mean_roc_auc}')


NotFittedError: This MultiOutputClassifier instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [9]:
test_data = pd.read_csv('path/to/your/test_data.csv')
X_test = test_data.drop(columns=['respondent_id'])
y_test_proba = clf.predict_proba(X_test)
y_test_proba = pd.DataFrame({col: [proba[:, 1] for proba in y_test_proba[i]] for i, col in enumerate(['xyz_vaccine', 'seasonal_vaccine'])})

submission = test_data[['respondent_id']].copy()
submission['xyz_vaccine'] = y_test_proba['xyz_vaccine']
submission['seasonal_vaccine'] = y_test_proba['seasonal_vaccine']

submission.to_csv('submission.csv', index=False)


FileNotFoundError: [Errno 2] No such file or directory: 'path/to/your/test_data.csv'